In [1]:
import time

"""
Sudoku Solver by Evan Freeman
Brute force-ish


Input must be a string of 81 characters
Blank cells may be filled with anything for the input.



For example:
    .94...13..............76..2.8..1.....32.........2...6.....5.4.......8..7..63.4..8
    
Which solves to:
    794582136268931745315476982689715324432869571157243869821657493943128657576394218


Here's the coordinates of every cell in the grid:

(0,0) (0,1) (0,2) (0,3) (0,4) (0,5) (0,6) (0,7) (0,8)
(1,0) (1,1) (1,2) (1,3) (1,4) (1,5) (1,6) (1,7) (1,8)
(2,0) (2,1) (2,2) (2,3) (2,4) (2,5) (2,6) (2,7) (2,8)
(3,0) (3,1) (3,2) (3,3) (3,4) (3,5) (3,6) (3,7) (3,8)
(4,0) (4,1) (4,2) (4,3) (4,4) (4,5) (4,6) (4,7) (4,8)
(5,0) (5,1) (5,2) (5,3) (5,4) (5,5) (5,6) (5,7) (5,8)
(6,0) (6,1) (6,2) (6,3) (6,4) (6,5) (6,6) (6,7) (6,8)
(7,0) (7,1) (7,2) (7,3) (7,4) (7,5) (7,6) (7,7) (7,8)
(8,0) (8,1) (8,2) (8,3) (8,4) (8,5) (8,6) (8,7) (8,8)


Big picture:
1) start filling in blanks, going from left to right, with 1s
    -so, make a list of coordinates that are not yet filled
    -traverse back and forth along this list, filling in, until we get to the end!
2) keep track of each step we take, both the value we gave it and the index of that step
3) check for consistency after each addition
    - better to just check the relevant row, column, and box, but we could check the whole thing in a pinch
4) If there is a contradiction, step backwards once an increment that guess. If that guess is at 9, step back again and increment that guess.
    -continue stepping back until we reach a step which was not 9, then continue from there.
5) keep going until we don't have a contradiction!!
    -Well, also check that we're full somehow
    -This must be A solution
    -My method won't check if there's multiple solutions, though it could...

To do:
    1) Accept more input types
    2) Check for multiple solutions
    3) Implement sudoku strategies to speed up solve time:
        -Could still finish off with brute force, even if I just put in a few basic strategies
        -Do the 'little numbers' technique, where I keep track of the possibilites associated with each cell
    4) Make it display as it goes?
        -That means showing MILLIONS of iterations. Would have to animate SUPER FAST.
        -Or collect all those images and make a gif of it, after the fact.
            - Pre-complile??? I don't know what I'm talking about
    5) Just how brute force is my algorithm? I think it's slightly faster than just populating every blank with a guess,
        checking the whole puzzle, then updating the whole puzzle and checking again (or even just checking the parts effected
        by the update).
        -After all, my algorithm cuts off certian possibility spaces as it goes
        -Still pretty brute force

"""

start_time = time.time()

class Grid:

    def __init__(self, string):
        self.cells = [[x for x in string[0:9]],
                      [x for x in string[9:18]],
                      [x for x in string[18:27]],
                      [x for x in string[27:36]],
                      [x for x in string[36:45]],
                      [x for x in string[45:54]],
                      [x for x in string[54:63]],
                      [x for x in string[63:72]],
                      [x for x in string[72:81]]]
                                    
          
    # This function outputs the contents of the box containing the cell with coordinates i, j
    def box(self, i, j):
        #let's find the coordinate of the upper left cell in the box
        #We'll calculate the rest of the cell from there
        
        #box x coordinate
        x = i // 3 * 3
        #box y coordinate
        y = j // 3 * 3
        
        box = [self.cells[a][b] for a in [x, x+1, x+2] for b in [y, y+1, y+2]]
        return box
        
    
    #This function outputs the contents of the row containing the cell with coordinates i, j
    def row(self, i, j):
        row = [self.cells[i][y] for y in range(9)]
        return row
    
    
    #This function outputs the contents of the column containing the cell with coordinates i, j
    def column(self, i, j):
        column = [self.cells[x][j] for x in range(9)]
        return column
    
    
    #Displays the puzzle, as a single block of strings
    def display(self):
        print('')
        for x in self.cells:
            print(''.join(x))
        print('')
    
    
    #Displays the puzzle, broken up into lists
    def display_grid(self):
        print('')
        for x in self.cells:
            print(x)
        print('')
     
        
        
#Checks a given input (box, row, or column) for duplicates
#Could be any list really, but will only check for duplicates in 1-9 (As strings)
#Should this be part of the sudoku object? Doesn't act on the object, so I don't think so
def check(thing):
    #First, remove any empty spaces
    clean_thing = []
    for x in thing:
        if x in ['1', '2', '3', '4', '5', '6', '7', '8', '9']:
            clean_thing.append(x)
    #Now check for duplicates
    if len(clean_thing) == len(set(clean_thing)):
        return True
    else:
        return False

#Here is the solution function. Takes us from the original puzzle to the solution.
#
def solve(puzzle):
    sudoku = Grid(puzzle)

    sudoku.display()      

    
    #Step 1: #First, generate a list of all blank spaces, along with their coordinates, in the format of ['.', i, j]
    b = []
    for i in range(9):
        for j in range(9):
            if sudoku.cells[i][j] not in ['1', '2', '3', '4', '5', '6', '7', '8', '9']:
                #so keep track of the cell we are going to fill, and it's coordinates
                b.append([sudoku.cells[i][j], i, j])

    #Initialize some variables
    i = 0    
    count = 0 

    #This is the engine that drives the solution 
    #In each scenario, we update both the list of blanks, and the sudoku grid itself
    #Keep going until our index hits the length of blanks (Which is to say, we're one step beyond)
    while i != len(b):
        count += 1
        
        #Scenario 1: blank number i is still blank. Start with 1
        if b[i][0] not in ['1', '2', '3', '4', '5', '6', '7', '8', '9']:
            b[i][0] = '1'
            sudoku.cells[b[i][1]][b[i][2]] = b[i][0]
        
        #Scenario 2: blank number i is at 9. So we've already tried all the options
        #So we need to clear it out and step back.
        #Also we skip the rest of the loop, becacuse we don't need to check for consistency
        #In fact, it would be bad to check for consistency, as we are guarenteed to trivially be consistent
        #This would lead to stepping forward, canceling out our step back, and ending up in an infinite loop
        elif b[i][0] == '9':
            b[i][0] = '.'
            sudoku.cells[b[i][1]][b[i][2]] = b[i][0]
            i -= 1
            continue
        
        #Scenario 3: There's some number 1-8 already plugged in. So we step forward by one.
        else:
            b[i][0] = str(int(b[i][0]) + 1)
            sudoku.cells[b[i][1]][b[i][2]] = b[i][0]
        
        #Now we check for consistency. If we are consistent, we'll step forward.
        #If not, we'll run through this same spot again.
        consistent = check(sudoku.row(b[i][1], b[i][2])) and check(sudoku.column(b[i][1], b[i][2])) and check(sudoku.box(b[i][1], b[i][2]))
        if consistent:
            i += 1

    #Format the solution as a string of 81 characters, like the input
    solution = ''.join([''.join(x) for x in sudoku.cells])

    sudoku.display()
    print(solution)
    print('\n')
    print(f'This sudoku was solved in {count} loops.')
    print('\n')
    print("--- %s seconds ---" % (time.time() - start_time))
    return solution


# I've added a particular puzzle for it to solve. You can uncomment the lines below to make it ask for an input, 
# Or just plug in your own puzzle below.

# puzzle = input('Please input your sudoku puzzle as a string of 81 characters. ')      
#solve(puzzle)

solve('.94...13..............76..2.8..1.....32.........2...6.....5.4.......8..7..63.4..8')    



.94...13.
.........
....76..2
.8..1....
.32......
...2...6.
....5.4..
.....8..7
..63.4..8


794582136
268931745
315476982
689715324
432869571
157243869
821657493
943128657
576394218

794582136268931745315476982689715324432869571157243869821657493943128657576394218


This sudoku was solved in 2329276 loops.


--- 11.77097773551941 seconds ---


'794582136268931745315476982689715324432869571157243869821657493943128657576394218'

In [2]:
solve('.94...13..............76..2.8..1.....32.........2...6.....5.4.......8..7..63.4..8')


.94...13.
.........
....76..2
.8..1....
.32......
...2...6.
....5.4..
.....8..7
..63.4..8


794582136
268931745
315476982
689715324
432869571
157243869
821657493
943128657
576394218

794582136268931745315476982689715324432869571157243869821657493943128657576394218


This sudoku was solved in 2329276 loops.


--- 23.668402910232544 seconds ---


'794582136268931745315476982689715324432869571157243869821657493943128657576394218'

In [3]:
solve('............942.8.16.....29........89.6.....14..25......4.......2...8.9..5....7..')


.........
...942.8.
16.....29
........8
9.6.....1
4..25....
..4......
.2...8.9.
.5....7..


249186573
735942186
168375429
512697348
976834251
483251967
694723815
327518694
851469732

249186573735942186168375429512697348976834251483251967694723815327518694851469732


This sudoku was solved in 317144 loops.


--- 25.227720975875854 seconds ---


'249186573735942186168375429512697348976834251483251967694723815327518694851469732'

In [4]:
solve('.....7....9...1.......45..6....2.....36...41.5.....8.9........4....18....815...32')


.....7...
.9...1...
....45..6
....2....
.36...41.
5.....8.9
........4
....18...
.815...32


653287941
794631258
128945376
819724563
236859417
547163829
965372184
372418695
481596732

653287941794631258128945376819724563236859417547163829965372184372418695481596732


This sudoku was solved in 54770833 loops.


--- 283.1789984703064 seconds ---


'653287941794631258128945376819724563236859417547163829965372184372418695481596732'

In [5]:
solve('......24...2.85..1.9..42.........5.26..4..3.9...1...68..5....8.7..5...9...97.4...')


......24.
..2.85..1
.9..42...
......5.2
6..4..3.9
...1...68
..5....8.
7..5...9.
..97.4...


583971246
472685931
196342857
941863572
658427319
237159468
315296784
724518693
869734125

583971246472685931196342857941863572658427319237159468315296784724518693869734125


This sudoku was solved in 82280 loops.


--- 283.5751314163208 seconds ---


'583971246472685931196342857941863572658427319237159468315296784724518693869734125'

In [6]:
#Hard 1

solve('4.....8.5.3..........7......2.....6.....8.4......1.......6.3.7.5..2.....1.4......')


4.....8.5
.3.......
...7.....
.2.....6.
....8.4..
....1....
...6.3.7.
5..2.....
1.4......


417369825
632158947
958724316
825437169
791586432
346912758
289643571
573291684
164875293

417369825632158947958724316825437169791586432346912758289643571573291684164875293


This sudoku was solved in 97273649 loops.


--- 796.1523821353912 seconds ---


'417369825632158947958724316825437169791586432346912758289643571573291684164875293'

In [7]:
#Hard 2

solve('52...6.........7.13...........4..8..6......5...........418.........3..2...87.....')


52...6...
......7.1
3........
...4..8..
6......5.
.........
.418.....
....3..2.
..87.....


527316489
896542731
314987562
172453896
689271354
453698217
941825673
765134928
238769145

527316489896542731314987562172453896689271354453698217941825673765134928238769145


This sudoku was solved in 32525485 loops.


--- 964.728554725647 seconds ---


'527316489896542731314987562172453896689271354453698217941825673765134928238769145'

In [8]:


6.....8.3.4.7.................5.4.7.3..2.....1.6.......2.....5.....8.6......1....
48.3............71.2.......7.5....6....2..8.............1.76...3.....4......5....
....14....3....2...7..........9...3.6.1.............8.2.....1.4....5.6.....7.8...
......52..8.4......3...9...5.1...6..2..7........3.....6...1..........7.4.......3.
6.2.5.........3.4..........43...8....1....2........7..5..27...........81...6.....
.524.........7.1..............8.2...3.....6...9.5.....1.6.3...........897........
6.2.5.........4.3..........43...8....1....2........7..5..27...........81...6.....
.923.........8.1...........1.7.4...........658.........6.5.2...4.....7.....9.....
6..3.2....5.....1..........7.26............543.........8.15........4.2........7..
.6.5.1.9.1...9..539....7....4.8...7.......5.8.817.5.3.....5.2............76..8...
..5...987.4..5...1..7......2...48....9.1.....6..2.....3..6..2.......9.7.......5..
3.6.7...........518.........1.4.5...7.....6.....2......2.....4.....8.3.....5.....
1.....3.8.7.4..............2.3.1...........958.........5.6...7.....8.2...4.......
6..3.2....4.....1..........7.26............543.........8.15........4.2........7..
....3..9....2....1.5.9..............1.2.8.4.6.8.5...2..75......4.1..6..3.....4.6.
45.....3....8.1....9...........5..9.2..7.....8.........1..4..........7.2...6..8..
.237....68...6.59.9.....7......4.97.3.7.96..2.........5..47.........2....8.......
..84...3....3.....9....157479...8........7..514.....2...9.6...2.5....4......9..56
.98.1....2......6.............3.2.5..84.........6.........4.8.93..5...........1..
..247..58..............1.4.....2...9528.9.4....9...1.........3.3....75..685..2...
4.....8.5.3..........7......2.....6.....5.4......1.......6.3.7.5..2.....1.9......
.2.3......63.....58.......15....9.3....7........1....8.879..26......6.7...6..7..4
1.....7.9.4...72..8.........7..1..6.3.......5.6..4..2.........8..53...7.7.2....46
4.....3.....8.2......7........1...8734.......6........5...6........1.4...82......
.......71.2.8........4.3...7...6..5....2..3..9........6...7.....8....4......5....
6..3.2....4.....8..........7.26............543.........8.15........8.2........7..
.47.8...1............6..7..6....357......5....1..6....28..4.....9.1...4.....2.69.
......8.17..2........5.6......7...5..1....3...8.......5......2..4..8....6...3....
38.6.......9.......2..3.51......5....3..1..6....4......17.5..8.......9.......7.32
...5...........5.697.....2...48.2...25.1...3..8..3.........4.7..13.5..9..2...31..
.2.......3.5.62..9.68...3...5..........64.8.2..47..9....3.....1.....6...17.43....
.8..4....3......1........2...5...4.69..1..8..2...........3.9....6....5.....2.....
..8.9.1...6.5...2......6....3.1.7.5.........9..4...3...5....2...7...3.8.2..7....4
4.....5.8.3..........7......2.....6.....5.8......1.......6.3.7.5..2.....1.8......
1.....3.8.6.4..............2.3.1...........958.........5.6...7.....8.2...4.......
1....6.8..64..........4...7....9.6...7.4..5..5...7.1...5....32.3....8...4........
249.6...3.3....2..8.......5.....6......2......1..4.82..9.5..7....4.....1.7...3...
...8....9.873...4.6..7.......85..97...........43..75.......3....3...145.4....2..1
...5.1....9....8...6.......4.1..........7..9........3.8.....1.5...2..4.....36....
......8.16..2........7.5......6...2..1....3...8.......2......7..3..8....5...4....
.476...5.8.3.....2.....9......8.5..6...1.....6.24......78...51...6....4..9...4..7
.....7.95.....1...86..2.....2..73..85......6...3..49..3.5...41724................
.4.5.....8...9..3..76.2.....146..........9..7.....36....1..4.5..6......3..71..2..
.834.........7..5...........4.1.8..........27...3.....2.6.5....5.....8........1..
..9.....3.....9...7.....5.6..65..4.....3......28......3..75.6..6...........12.3.8
.26.39......6....19.....7.......4..9.5....2....85.....3..2..9..4....762.........4
2.3.8....8..7...........1...6.5.7...4......3....1............82.5....6...1.......
6..3.2....1.....5..........7.26............843.........8.15........8.2........7..
1.....9...64..1.7..7..4.......3.....3.89..5....7....2.....6.7.9.....4.1....129.3.
.........9......84.623...5....6...453...1...6...9...7....1.....4.5..2....3.8....9
.2....5938..5..46.94..6...8..2.3.....6..8.73.7..2.........4.38..7....6..........5
9.4..5...25.6..1..31......8.7...9...4..26......147....7.......2...3..8.6.4.....9.
...52.....9...3..4......7...1.....4..8..453..6...1...87.2........8....32.4..8..1.
53..2.9...24.3..5...9..........1.827...7.........981.............64....91.2.5.43.
1....786...7..8.1.8..2....9........24...1......9..5...6.8..........5.9.......93.4
....5...11......7..6.....8......4.....9.1.3.....596.2..8..62..7..7......3.5.7.2..
.47.2....8....1....3....9.2.....5...6..81..5.....4.....7....3.4...9...1.4..27.8..
......94.....9...53....5.7..8.4..1..463...........7.8.8..7.....7......28.5.26....
.2......6....41.....78....1......7....37.....6..412....1..74..5..8.5..7......39..
1.....3.8.6.4..............2.3.1...........758.........7.5...6.....8.2...4.......
2....1.9..1..3.7..9..8...2.......85..6.4.........7...3.2.3...6....5.....1.9...2.5
..7..8.....6.2.3...3......9.1..5..6.....1.....7.9....2........4.83..4...26....51.
...36....85.......9.4..8........68.........17..9..45...1.5...6.4....9..2.....3...
34.6.......7.......2..8.57......5....7..1..2....4......36.2..1.......9.......7.82
......4.18..2........6.7......8...6..4....3...1.......6......2..5..1....7...3....
.4..5..67...1...4....2.....1..8..3........2...6...........4..5.3.....8..2........
.......4...2..4..1.7..5..9...3..7....4..6....6..1..8...2....1..85.9...6.....8...3
8..7....4.5....6............3.97...8....43..5....2.9....6......2...6...7.71..83.2
.8...4.5....7..3............1..85...6.....2......4....3.26............417........
....7..8...6...5...2...3.61.1...7..2..8..534.2..9.......2......58...6.3.4...1....
......8.16..2........7.5......6...2..1....3...8.......2......7..4..8....5...3....
.2..........6....3.74.8.........3..2.8..4..1.6..5.........1.78.5....9..........4.
.52..68.......7.2.......6....48..9..2..41......1.....8..61..38.....9...63..6..1.9
....1.78.5....9..........4..2..........6....3.74.8.........3..2.8..4..1.6..5.....
1.......3.6.3..7...7...5..121.7...9...7........8.1..2....8.64....9.2..6....4.....
4...7.1....19.46.5.....1......7....2..2.3....847..6....14...8.6.2....3..6...9....
......8.17..2........5.6......7...5..1....3...8.......5......2..3..8....6...4....
963......1....8......2.5....4.8......1....7......3..257......3...9.2.4.7......9..
15.3......7..4.2....4.72.....8.........9..1.8.1..8.79......38...........6....7423
..........5724...98....947...9..3...5..9..12...3.1.9...6....25....56.....7......6
....75....1..2.....4...3...5.....3.2...8...1.......6.....1..48.2........7........
6.....7.3.4.8.................5.4.8.7..2.....1.3.......2.....5.....7.9......1....
....6...4..6.3....1..4..5.77.....8.5...8.....6.8....9...2.9....4....32....97..1..
.32.....58..3.....9.428...1...4...39...6...5.....1.....2...67.8.....4....95....6.
...5.3.......6.7..5.8....1636..2.......4.1.......3...567....2.8..4.7.......2..5..
.5.3.7.4.1.........3.......5.8.3.61....8..5.9.6..1........4...6...6927....2...9..
..5..8..18......9.......78....4.....64....9......53..2.6.........138..5....9.714.
..........72.6.1....51...82.8...13..4.........37.9..1.....238..5.4..9.........79.
...658.....4......12............96.7...3..5....2.8...3..19..8..3.6.....4....473..
.2.3.......6..8.9.83.5........2...8.7.9..5........6..4.......1...1...4.22..7..8.9
.5..9....1.....6.....3.8.....8.4...9514.......3....2..........4.8...6..77..15..6.
.....2.......7...17..3...9.8..7......2.89.6...13..6....9..5.824.....891..........
3...8.......7....51..............36...2..4....7...........6.13..452...........8..

SyntaxError: invalid syntax (<ipython-input-8-442f002296b9>, line 1)